In [ ]:
#Required Liberyes 
import sys
import os
import keras
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import Xception 
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from time import time

# from tensorflow.python.keras.layers import Input, Dense
# from tensorflow.python.keras.models import Sequential

sys.path.append(r'C:\Users\venkatareddy.m\Deep Learning\\Rust_VGG')
import tensorflow as tf



In [ ]:
base_model = VGG16(weights='imagenet',include_top=False,input_shape=(150, 150, 3))
base_model.summary()
# base_model = ResNet50(weights='imagenet',include_top=False,input_shape=(150, 150, 3))
# base_model.summary()

In [ ]:
def prototyping():
    prototype = models.Sequential()
    prototype.add(base_model)
    prototype.add(layers.Flatten()) #Flattening is converting the data into a 1-dimensional array for inputting it to the next layer.
    prototype.add(layers.Dense(256, activation='relu')) #dense layer is a layer that is deeply connected with its preceding layer
    prototype.add(layers.Dense(1, activation='sigmoid'))
    prototype.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])
    prototype.summary()
    return prototype

In [ ]:
def train_model(): 
    train_loc = r".\Input\train"
    validation_loc = r".\Input\validation"
    train_dataset = ImageDataGenerator(rescale=1./255,rotation_range=30,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,fill_mode='nearest')
    test_dataset = ImageDataGenerator(rescale=1./255)
    trainner = train_dataset.flow_from_directory(train_loc,target_size=(150, 150),batch_size=4,class_mode='binary')
    validator = test_dataset.flow_from_directory(validation_loc,target_size=(150, 150),batch_size=16,class_mode='binary')
    return trainner, validator

In [ ]:
model = prototyping()
trainner, validator = train_model()

In [ ]:
# csv_logger = tf.keras.callbacks.CSVLogger("log.csv", separator=",", append=True)
tensor_board = keras.callbacks.TensorBoard(log_dir='output/{}'.format(time()))
model.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.RMSprop(learning_rate=2e-5),metrics=['acc'])

In [ ]:
model.fit(trainner,steps_per_epoch=10,epochs=10,validation_data=validator,validation_steps=20,verbose=2,callbacks=[tensor_board])

In [ ]:
model.save('my_model_final.h5')

In [ ]:
from keras.preprocessing import image
from keras.models import load_model


def main_module():
    model = load_model('my_model_final.h5')
#     image_path = r'.\Input\test\rust\000016.jpg'
    image_path = r'.\Input\test\norust\norust.4.jpg'
    input_image = image.load_img(image_path, target_size=(150, 150))
    image_test = image.img_to_array(input_image)
    image_test = image_test.reshape((1,) + image_test.shape)
    image_test = image_test.astype('float32') / 255
    rust_prob = model.predict(image_test)
    if (rust_prob > 0.50):
        print("This is a Rust image")
    else:
        print("This is a no Rust image")

    return


main_module()